Preliminary Data Exploration

In [1]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
import pandas as pd
import numpy as np

project_dir = '../'
scripts_dir = os.path.join(project_dir, 'scripts')
sys.path.append(scripts_dir)

from tlc_data_collect import collect_tlc_yellow_data
from tlc_data_collect import collect_tlc_green_data


In [2]:
collect_tlc_yellow_data()

Begin month 07
Completed month 07
Begin month 08
Completed month 08
Begin month 09
Completed month 09
Begin month 10
Completed month 10
Begin month 11
Completed month 11
Begin month 12
Completed month 12


In [3]:
collect_tlc_green_data()

Begin month 07
Completed month 07
Begin month 08
Completed month 08
Begin month 09
Completed month 09
Begin month 10
Completed month 10
Begin month 11
Completed month 11
Begin month 12
Completed month 12


In [4]:
yellow_df = pd.read_parquet('../data/raw/tlc_data/yellow')
green_df = pd.read_parquet('../data/raw/tlc_data/green')

In [5]:
yellow_df.columns


Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [6]:
green_df.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')

In [7]:
columns_yellow_df = set(yellow_df.columns)
columns_green_df = set(green_df.columns)

# Find differences
only_in_yellow_df = columns_yellow_df - columns_green_df
only_in_green_df = columns_green_df - columns_yellow_df
in_both = columns_yellow_df & columns_green_df

print(f"Columns only in df1: {only_in_yellow_df}")
print(f"Columns only in df2: {only_in_green_df}")
print(f"Columns in both df1 and df2: {in_both}")

Columns only in df1: {'tpep_dropoff_datetime', 'Airport_fee', 'tpep_pickup_datetime'}
Columns only in df2: {'lpep_dropoff_datetime', 'lpep_pickup_datetime', 'ehail_fee', 'trip_type'}
Columns in both df1 and df2: {'RatecodeID', 'PULocationID', 'payment_type', 'VendorID', 'tip_amount', 'trip_distance', 'mta_tax', 'DOLocationID', 'store_and_fwd_flag', 'congestion_surcharge', 'passenger_count', 'extra', 'tolls_amount', 'total_amount', 'fare_amount', 'improvement_surcharge'}


In [8]:
yellow_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-07-01 00:29:59,2023-07-01 00:40:15,1.0,1.80,1.0,N,140,263,1,12.1,3.5,0.5,5.10,0.0,1.0,22.20,2.5,0.0
1,2,2023-07-01 00:03:25,2023-07-01 00:23:44,1.0,2.31,1.0,N,163,163,2,19.1,1.0,0.5,0.00,0.0,1.0,24.10,2.5,0.0
2,2,2023-07-01 00:38:29,2023-07-01 00:48:53,1.0,2.36,1.0,N,142,262,1,13.5,1.0,0.5,3.70,0.0,1.0,22.20,2.5,0.0
3,2,2023-07-01 00:14:16,2023-07-01 00:29:13,1.0,4.36,1.0,N,68,24,1,19.8,1.0,0.5,4.96,0.0,1.0,29.76,2.5,0.0
4,1,2023-07-01 00:11:15,2023-07-01 00:20:47,0.0,1.60,1.0,N,161,107,1,11.4,3.5,0.5,3.25,0.0,1.0,19.65,2.5,0.0


In [9]:
green_df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-07-01 00:08:06,2023-07-01 00:21:24,N,1.0,75,170,2.0,3.45,17.0,1.0,0.5,4.45,0.0,NaN,1.0,26.70,1.0,1.0,2.75
1,2,2023-07-01 00:56:14,2023-07-01 01:03:28,N,1.0,75,262,2.0,1.49,10.0,1.0,0.5,3.05,0.0,NaN,1.0,18.30,1.0,1.0,2.75
2,2,2023-06-30 23:47:32,2023-06-30 23:47:35,N,5.0,42,42,2.0,0.00,50.0,0.0,0.0,15.30,0.0,NaN,1.0,66.30,1.0,2.0,0.00
3,2,2023-07-01 00:30:12,2023-07-01 00:51:20,N,1.0,33,75,1.0,9.32,38.0,1.0,0.5,0.00,0.0,NaN,1.0,43.25,2.0,1.0,2.75
4,2,2023-07-01 00:46:56,2023-07-01 00:49:02,N,1.0,74,74,1.0,0.97,5.8,1.0,0.5,0.00,0.0,NaN,1.0,8.30,1.0,1.0,0.00


['tpep_pickup_datetime','tpep_dropoff_datetime','passenger_count','trip_distance','PULocationID','DOLocationID','payment_type','fare_amount','tip_amount','total_amount']

tpep_pickup_datetime & tpep_dropoff_datetime: Capture the timing and duration of trips, crucial for understanding demand patterns and peak usage times.

Passenger count: Provides insights into ride-sharing behavior and vehicle occupancy, which are important for capacity planning, optimizing fleet usage, and understanding the demand for different vehicle sizes or services.

trip_distance: Key metric for evaluating trip lengths, fare calculation, and understanding travel behavior.

PULocationID & DOLocationID: Identify pick-up and drop-off locations, essential for analyzing trip patterns and spatial distribution of rides.

payment_type: Helps in analyzing payment trends and customer preferences, impacting business decisions.

fare_amount: Core component of revenue analysis and pricing strategies.

tip_amount: Reflects customer satisfaction and service quality, useful for evaluating driver performance.

total_amount: Overall revenue per trip, essential for financial analysis and business forecasting.


In [10]:
import os

processed_dir = '../data/curated'
os.makedirs(processed_dir, exist_ok=True)

columns_to_select_y = [
    'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 
    'trip_distance', 'PULocationID', 'DOLocationID', 'payment_type', 
    'fare_amount', 'tip_amount', 'total_amount'
]
sub_ydf = yellow_df.loc[:, columns_to_select_y]
print(sub_ydf.head())

parquet_file_path = os.path.join(processed_dir, 'tlc_yellow.parquet')
sub_ydf.to_parquet(parquet_file_path, compression='gzip')


  tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  trip_distance  \
0  2023-07-01 00:29:59   2023-07-01 00:40:15              1.0           1.80   
1  2023-07-01 00:03:25   2023-07-01 00:23:44              1.0           2.31   
2  2023-07-01 00:38:29   2023-07-01 00:48:53              1.0           2.36   
3  2023-07-01 00:14:16   2023-07-01 00:29:13              1.0           4.36   
4  2023-07-01 00:11:15   2023-07-01 00:20:47              0.0           1.60   

   PULocationID  DOLocationID  payment_type  fare_amount  tip_amount  \
0           140           263             1         12.1        5.10   
1           163           163             2         19.1        0.00   
2           142           262             1         13.5        3.70   
3            68            24             1         19.8        4.96   
4           161           107             1         11.4        3.25   

   total_amount  
0         22.20  
1         24.10  
2         22.20  
3         29.7

In [11]:
columns_to_select_g = [
    'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'passenger_count', 
    'trip_distance', 'PULocationID', 'DOLocationID', 'payment_type', 
    'fare_amount', 'tip_amount', 'total_amount'
]

sub_gdf = green_df.loc[:, columns_to_select_g]
print(sub_gdf.head())

parquet_file_path = os.path.join(processed_dir, 'tlc_green.parquet')
sub_gdf.to_parquet(parquet_file_path, compression='gzip')

  lpep_pickup_datetime lpep_dropoff_datetime  passenger_count  trip_distance  \
0  2023-07-01 00:08:06   2023-07-01 00:21:24              2.0           3.45   
1  2023-07-01 00:56:14   2023-07-01 01:03:28              2.0           1.49   
2  2023-06-30 23:47:32   2023-06-30 23:47:35              2.0           0.00   
3  2023-07-01 00:30:12   2023-07-01 00:51:20              1.0           9.32   
4  2023-07-01 00:46:56   2023-07-01 00:49:02              1.0           0.97   

   PULocationID  DOLocationID  payment_type  fare_amount  tip_amount  \
0            75           170           1.0         17.0        4.45   
1            75           262           1.0         10.0        3.05   
2            42            42           1.0         50.0       15.30   
3            33            75           2.0         38.0        0.00   
4            74            74           1.0          5.8        0.00   

   total_amount  
0         26.70  
1         18.30  
2         66.30  
3         43.2